
# Prompt Engineering

Prompt engineering is using techniques when writing prompts to get better, and more consistent, responses from AI models like ChatGPT and Claude.  The more complex task the more benefit you will get from prompt engineering.


**Prompt engineering techniques:**

- give clear instructions
- provide examples
- set a persona
- start a new chat if things go wrong

**Advanced techniques:**

- XML tagging
- Give the model time to think/ Break up complex tasks
- Use format support for output consistency
- Prefilled responses
- Cater for model context limits

**Links:**

https://platform.openai.com/docs/guides/prompt-engineering/prompt-engineering

https://docs.anthropic.com/claude/docs/prompt-engineering

https://docs.ell.so/


**Setup:**

For this session you will need an OpenAI account, and an OpenAI API token setup. https://platform.openai.com/settings then click on "API keys" where you can create a new key.

You will need a .env file in the project directory with the following line filled out
> OPENAI_API_KEY = \<your-openai-api-key\>


In [ ]:
%pip install python-dotenv
%pip install openai --upgrade
%pip install -U ell-ai

from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()


If you have issues install the above packages with a message about a missing 'typing_extgensions' package then try running these on the command line and then try again
```
pip install --upgrade pip
pip install typing-extensions
pip install ipykernel --upgrade --force-reinstall
python -m ipykernel install --user
```

# Give clear instructions

Give details, be specific, assume that the model doesn't know what you are talking about.


In [3]:
# Simple function that sends ChatGPT user prompt requests
def simple_prompt(prompt):
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content.strip()


""" 
Examples of being more specific in your prompting 
"""
my_prompt = "what is the cheapest car to own in Australia?"
# my_prompt = "what is the cheapest petrol car to own in Australia? The car model should not be more than 5 years old and have a 5 star safety rating. Exclude SUVs or anything with 4WD. Give the estimated price in Australian dollars and the year of manufacture"

# my_prompt = "Who was Australia’s greatest prime minister?"
# my_prompt = "Who was Australia’s greatest prime minister?  The answer is subjective but if you had to pick one who would it be and why?"

print(simple_prompt(my_prompt))


The cheapest car to own in Australia can vary based on factors such as purchase price, fuel efficiency, maintenance costs, insurance, and depreciation. As of the latest available information, the Suzuki Alto, Kia Picanto, and Mitsubishi Mirage are often cited as some of the cheapest cars to own due to their low purchase prices and good fuel economy.

When determining the cheapest car to own, it's important to consider not only the initial purchase price but also ongoing expenses. It's advisable to look at annual running costs and consult recent reviews or cost comparisons from trusted Australian automotive websites or financial institutions that analyze these factors comprehensively.


# Give examples

LLM's **LOVE** examples. 

This could include:
- examples on how you want the model to repond
- example on how you want the reponse to be formatted (e.g. json) **

** this can also be done througn Pydantic Python


### Number of examples given: 

The more examples you give the better, and this technique has names.

**Zero-shot:**
- no examples given
- simple but less accurate for complex tasks

**One-shot:**
- one example
- helps model understand required response/format
- even one example is significantly better than none

**Few-shot/Many-shot:**
- multiple examples given
- better performance as more context for model to work with
- excels for use with complex tasks/specific formats (input and/or output)



In [4]:
# Example of defining the output you want to receive

my_prompt = """ You are an expert in sports statistics. List the top three Australian female tennis players in 2022. Include their win/loss results and total estimated prise money for that year.
Output results in a numbered list with the following format:
John McEnroe, ranking:3rd, prize money: $1,234,000, win/loss: 45/13
"""

print(simple_prompt(my_prompt))

As of 2022, the top Australian female tennis players, along with their win/loss records and estimated prize money, are as follows:

1. Ashleigh Barty, ranking: 1st, prize money: Approximately $2,500,000, win/loss: 14/1
2. Ajla Tomljanović, ranking: 33rd, prize money: Approximately $1,026,000, win/loss: 36/26
3. Daria Saville, ranking: 53rd, prize money: Approximately $600,000, win/loss: 23/18

Note: Rankings, prize money, and win/loss records can vary slightly depending on the source, as these figures are estimated based on available data.


In [5]:
# Define a function that lets us specify a set of messages for our prompt
def complex_prompt(messages):
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=messages
    )
    
    return response.choices[0].message.content.strip()


# Example of giving no, one, or many examples in your prompt

input_text = "Love this thing"
#input_text = "did what it said it would do"
#input_text = "Mostly I liked it, but the price seemed difficult to accept"
#input_text = "it's a cracker"


# No-shot prompting - just the task with no examples
no_shot_msgs = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": f"Classify this text as either positive or negative: '{input_text}'"}
]

# One-shot prompting - includes a single example
one_shot_msgs = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Classify this text as either positive or negative: 'I love this product!'"},
    {"role": "assistant", "content": "Positive"},
    {"role": "user", "content": f"Classify this text as either positive or negative: '{input_text}'"}
]

# Many-shot prompting - includes multiple examples
many_shot_msgs = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Classify this text as either positive or negative: 'I love this product!'"},
    {"role": "assistant", "content": "Positive"},
    {"role": "user", "content": "Classify this text as either positive or negative: 'This is terrible.'"},
    {"role": "assistant", "content": "Negative"},
    {"role": "user", "content": "Classify this text as either positive or negative: 'It's okay, nothing special.'"},
    {"role": "assistant", "content": "Positive"},
    {"role": "user", "content": f"Classify this text as either positive or negative: '{input_text}'"}
]

print(f"no-shot: {complex_prompt(no_shot_msgs)}")
print("*" * 50)  # Prints 50 asterisks in a row
print(f"one-shot: {complex_prompt(one_shot_msgs)}")
print("*" * 50)  # Prints 50 asterisks in a row
print(f"many-shot: {complex_prompt(many_shot_msgs)}")

no-shot: Positive
**************************************************
one-shot: Positive
**************************************************
many-shot: Positive


# Set a persona or role

Setting a persona gives additional context to the response that should be generated.  Likewise you could describe the role or task the model is responsible for completing.

Examples:
- "You are an expert in \<subject\>" (marketing, HR, English grammer, etc), or "You are a \<position\>" (history teacher, mathematician, dietician, etc)
- "Your role is to \<role/task\> (identify common trends in datasets, make recommendations to improve processes, review content for grammar and appropriate reading comprehension levels, etc)


In [6]:
my_prompt = "You are an expert in marketing.  List the top 5 activities marketers should do to have a successful social media campaign."

# my_prompt = "You are an English woman born in the early 1900's. Describe your rights under the English legal system. Summarise in a bulleted list of no more than 5 points"

print(simple_prompt(my_prompt))

Certainly! A successful social media campaign requires careful planning, execution, and analysis. Here are the top five activities marketers should focus on:

1. **Define Clear Objectives**:
   - Establish what you want to achieve with your campaign, such as increasing brand awareness, driving website traffic, generating leads, or boosting sales. Clear objectives will guide your strategy and help measure the success of your campaign.

2. **Know Your Audience**:
   - Conduct thorough audience research to understand the demographics, preferences, behaviors, and pain points of your target audience. This information will help tailor your content and messaging to resonate with them effectively.

3. **Create Engaging and Relevant Content**:
   - Develop high-quality content that aligns with your brand and appeals to your audience. Use a mix of formats such as images, videos, infographics, and live streams to keep your audience engaged. Ensure that the content is authentic and provides value 

# Start a new chat

If things go wrong start new chat to avoid previous requests/responses being including in your prompt.  If using OpenAI assistants consider starting a new thread (threads automatically provide previous message history to the assistant).  

People will often talk of previous message history 'polluting' the current request and causing unwanted results.


---

Now onto more advanced techniques.

Advanced prompt engineering techniques should be used when:

- implementing complex prompts 

- we require specific and consistently formatted responses (e.g. JSON, etc).


# XML Tagging

To help the model to identify the different parts of a prompt (like where an example starts/ends) we can use delimters.  One useful type of delimiters are xml tags (e.g."\<code\>...\</code\>").  The name of text in the tag doesn't matter, although giving it a meaningful name will probably help (like "example", "code", "format", etc).

https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/use-xml-tags

In [7]:
# Here we specifically identify the example and instruction text for the model.
# There are better ways of getting models to return JSON formatted responses but
# it is a good example of clearly defining the different parts of the prompt.

movie_title = "Terminator 2"

my_prompt = """
    Provide information about the movie in JSON format. 
    
    <example>
    {
        "title": "The Matrix",
        "year": 1999,
        "director": "The Wachowskis",
        "rating": 8.7,
        "genres": ["Sci-Fi", "Action"],
        "mainCast": [
            {
                "name": "Keanu Reeves",
                "role": "Neo"
            },
            {
                "name": "Laurence Fishburne",
                "role": "Morpheus"
            }
        ]
    }
    </example>

    <instructions>
    Follow the exact same JSON structure as shown in the example above.
    Ensure all field names match exactly.
    Include only factual information about the movie.
    Include 3-5 cast members.
    </instructions>

    Provide information for the movie: {movie_title}
    """
    
messages = [
    {"role": "system", "content": "You are a movie database expert. Always return valid JSON."},
    {"role": "user", "content": my_prompt}
]

response = complex_prompt(messages)

print(response)


```json
{
    "title": "The Matrix",
    "year": 1999,
    "director": "The Wachowskis",
    "rating": 8.7,
    "genres": ["Sci-Fi", "Action"],
    "mainCast": [
        {
            "name": "Keanu Reeves",
            "role": "Neo"
        },
        {
            "name": "Laurence Fishburne",
            "role": "Morpheus"
        },
        {
            "name": "Carrie-Anne Moss",
            "role": "Trinity"
        },
        {
            "name": "Hugo Weaving",
            "role": "Agent Smith"
        }
    ]
}
```


# Give the model time to think 

One of the issues with LLM's is the importance of being responsive, in other words don't take too long to give a response.  However we may not care about how long the request takes if it means we get a better response.  Very recently OpenAI release o1 which now prioritises taking its time, breaking tasks into steps, and returning more meaningful results.

The key phrase to use in your prompt is "step by step".  Due to the way LLM's work you will always see evidence of the workings of a step, before it is feed into the next step, i.e. you will receive intermediate results before you get the final response.

https://docs.anthropic.com/claude/docs/let-claude-think


An alternative approach to to break you request into multiple smaller requests, feeding the output of previous request into the next.

In [8]:
# Example of telling the model to take its time in generating the response

my_prompt = f"""
    Bob has 2 tennis balls.  Be buys 2 more cans of tennis ball, each can holds 3 tennis balls.  How many tennis balls does Bob have now?
    Take your time. Think about this step-by-step and show all your work.
    """

    
messages = [
        {
            "role": "system", 
            "content": "You are a methodical problem solver who breaks down complex problems into manageable steps."
        },
        {
            "role": "user", 
            "content": my_prompt
        }
    ]

response = complex_prompt(messages)

print(response)

To solve the problem, we'll go through it step-by-step:

1. **Initial Number of Tennis Balls:**  
   Bob starts with 2 tennis balls.

2. **Number of Cans Bought:**  
   Bob buys 2 cans.

3. **Tennis Balls Per Can:**  
   Each can contains 3 tennis balls.

4. **Calculating Total Balls from the Cans:**  
   Since Bob buys 2 cans and each can contains 3 tennis balls, we calculate the total number of tennis balls in the cans as follows:
   \[
   \text{Total tennis balls from cans} = \text{Number of cans} \times \text{Tennis balls per can} = 2 \times 3 = 6
   \]

5. **Total Tennis Balls Overall:**  
   Finally, to find the total number of tennis balls Bob has, we add the number of tennis balls he originally had to the number of tennis balls from the cans:
   \[
   \text{Total tennis balls} = \text{Initial tennis balls} + \text{Total tennis balls from cans} = 2 + 6 = 8
   \]

Therefore, Bob now has 8 tennis balls.


# Breakup complex tasks

AI models can stuggle with large complex prompts and benefit from splitting them into smaller less complex tasks.  This can be approached two ways:

- Single task - maintain having a single prompt but give the AI a number of smaller steps to follow in constructing the response.

- Multiple tasks - break the prompt into a number of smaller, separate, prompts passing data between them were necessary (this technique is called “prompt chaining”)

Keep in mind large complex prompts may hit issues from the amount of time needed to process them (outlined in “Give the model time to think”). 

In [9]:
# Example defining the steps you want the model to follow in a single request
my_prompt = f"""
    I want you to solve this problem carefully and methodically. 
    
    <instructions>
    1. First, break down the problem into smaller parts
    2. For each part, explain your thinking process
    3. Show your work at each step
    4. Before giving the final answer, verify your solution
    5. If you make any assumptions, state them explicitly
    </instructions>

    <problem>
    A store is having a 20% off sale. If you buy a shirt that normally costs $80 
    and pants that normally cost $120, and you have a $25 coupon that applies after 
    the sale discount, what is your final total including 8% sales tax?
    </problem>

    Take your time. Think about this step-by-step and show all your work.
    """
    
messages = [
        {
            "role": "system", 
            "content": "You are a methodical problem solver who breaks down complex problems into manageable steps."
        },
        {
            "role": "user", 
            "content": my_prompt
        }
    ]

response = complex_prompt(messages)

print(response)

To solve this problem, we'll follow the instructions provided by breaking it into smaller parts and addressing each one methodically. Here is the complete breakdown:

### Step 1: Apply the Sale Discount

1. **Calculate the Discounted Price of Each Item:**

   - **Shirt:**
     - Original price = $80
     - Discount = 20% of $80
     - Discount = \( 0.20 \times 80 = 16 \)
     - Discounted price of shirt = $80 - $16 = $64

   - **Pants:**
     - Original price = $120
     - Discount = 20% of $120
     - Discount = \( 0.20 \times 120 = 24 \)
     - Discounted price of pants = $120 - $24 = $96

2. **Calculate the Total Price After Discounts:**
   - Total discounted price = $64 (shirt) + $96 (pants) = $160

### Step 2: Apply the $25 Coupon

1. **Subtract the Coupon Value from the Total Discounted Price:**

   - Total after coupon = $160 - $25 = $135

### Step 3: Calculate Sales Tax

1. **Determine the Sales Tax on the Total After the Coupon:**
   - Sales tax rate = 8%
   - Sales tax = \( 0

# Use format support for output consistency

Many LLM’s support different output formats like JSON, CSV, HTML, etc.  We can utililise this to ensure the output we receive from the LLM is in a consistent format for send to other jobs for processing.  This is extremely useful where we want to have the response in a consistant format we can pass to another process.

- certain key phrases like "JSON format" to trigger the functionality
- parameters in the request setting the format, like the response_format parameter in OpenAI API requests
- use of methods like Pydantic Python to set a structure/schema that the LLM recognises and conforms to.

(the later Ell example uses Pydantic Python to define the output structure)






# Prefilled responses

In this technique we have given the our user prompt as well as the start of the assitant response.  The model will start adding to the response it has been given, in this case "Once up Once upon a time, in a land far away, there was..."

In [10]:
messages = [
    {"role": "user", "content": "Write a very short story about a brave knight."},
    {"role": "assistant", "content": "Once upon a time, in a land far away, there was "}
]

reponse = complex_prompt(messages)

print(messages[1]['content'] + reponse)

Once upon a time, in a land far away, there was a brave knight named Sir Cedric. Known for his courage and unwavering sense of justice, Sir Cedric was beloved by all who dwelt in the kingdom of Eldoria.

One fateful day, a fierce dragon descended upon the peaceful village of Brookshire, casting a shadow of fear across the land. The villagers, having never faced such a peril, sent for Sir Cedric with hopes that he could rid them of this dreadful creature.

Without hesitation, Sir Cedric mounted his loyal steed, Onyx, and galloped towards the village. The journey was fraught with danger as the path wound through dark forests and steep mountains, yet he pressed on, unfazed. 

Upon reaching Brookshire, Sir Cedric saw the destruction the dragon had wrought. The villagers gathered around him, their eyes filled with both hope and terror. With a reassuring smile, he promised to protect them, and ventured into the dragon's lair alone.

As he approached, the dragon emerged, its scales glinting l

# Cater for model context limits
LLM's have a limit to the size of the request they can accept, and recently that limit has greatly increased (Claude 3.5 is now 200k tokens).  Hitting a context limit will either cause the request to error or the output to be truncated.  

Remember that the size of your request isn't just the prompt you wrote, but also any content (files, etc) you have attached.  The model may also automatically add in previous message history as well.


---

# Ell prompting framework

Recently the Ell framework has been released to help with developing prompts.  A few of its useful features are the versioning of prompts, keeping a history of requests/results, and a graphical front end.

Currently it supports the OpenAI and Anthropic APIs.  Here we are using the OpenAI API.


In [11]:
import ell
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

ell.init(store='./logdir', autocommit=True)

In [12]:
# traditional call using eh OpenAI API
# How we usually call LLM in OpenAI API
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Say hello to Sam Altman!"}
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages
)

print(response.choices[0].message.content)


Hello, Sam Altman! How can I assist you today?


In [13]:
# Now using Ell

@ell.simple(model="gpt-4o")
def hello(name: str):
    """You are a helpful assistant.""" # System prompt
    return f"Say hello to {name}!" # User prompt

hello("Sam Altman")

"Hello, Sam Altman! It's great to have you here. How can I assist you today?"

In [14]:
# You can enable verbose output to get more behind the scenes detail

ell.init(verbose=True)

print(hello("Bobby"))

ell.init(verbose=False)


╔══════════════════════════════════════════════════════════════════════════════╗
║ hello(Bobby)
╠══════════════════════════════════════════════════════════════════════════════╣
║ Prompt:
╟──────────────────────────────────────────────────────────────────────────────╢
│      system: You are a helpful assistant.
│
│        user: Say hello to Bobby!
╟──────────────────────────────────────────────────────────────────────────────╢
║ Output:
╟──────────────────────────────────────────────────────────────────────────────╢
│   assistant: Hello, Bobby! How are you today?
╚══════════════════════════════════════════════════════════════════════════════╝
Hello, Bobby! How are you today?


In [15]:
# Example of a more complex request with prefilled messages

@ell.simple(model="gpt-4o")
def hello(name: str):
    return [
        ell.system("You are a helpful assistant."),
        ell.user(f"Say hello to {name}!"),
        ell.assistant("Hello! I'd be happy to greet Sam Altman."),
        ell.user("Great! Now do it more enthusiastically.")
    ]

greeting = hello("Sam Altman")
print(greeting)

Hello, Sam Altman! It's fantastic to connect with you! 🎉


In [16]:
# As the prompt is effectively now a function we can easily call it
hello("Sara Smith")

"Hello, Sara Smith! 🌟 It's wonderful to meet you! I hope you're having an amazing day! 🎉😊"

In [17]:
# A slightly more useful example
@ell.simple(model="gpt-4o")
def tips(subject: str):
    return [
        ell.system("You are a helpful assistant."),
        ell.user(f"List 5 helpful tips for {subject}"),
    ]

response = tips("waking up on time")
print(response)

Waking up on time can be challenging, but with some strategies, you can make it easier. Here are five helpful tips:

1. **Establish a Consistent Sleep Schedule**: Go to bed and wake up at the same time every day, even on weekends. This helps regulate your body's internal clock and can make it easier to wake up naturally.

2. **Create a Bedtime Routine**: Develop a relaxing routine before bed to signal to your body that it's time to wind down. This could include reading a book, listening to calming music, or practicing meditation or deep breathing exercises.

3. **Limit Exposure to Screens Before Bed**: The blue light emitted by phones, tablets, and computers can interfere with your body's production of melatonin, a hormone that regulates sleep. Try to avoid screens at least an hour before bedtime.

4. **Optimize Your Sleep Environment**: Make sure your bedroom is conducive to sleep. Keep it cool, quiet, and dark, and invest in a comfortable mattress and pillows. Consider using blackout

In [18]:
print(tips("fixing cars"))

Certainly! Here are five helpful tips for fixing cars:

1. **Read the Manual:** Start by consulting your car's owner's manual. It contains valuable information about your vehicle's specific features and maintenance requirements. Understanding your car's layout and specifications will make diagnosing problems easier.

2. **Gather the Right Tools:** Make sure you have a basic set of automotive tools, including wrenches, screwdrivers, pliers, and a socket set. Specialized tools may sometimes be necessary depending on the repair, so identify what you'll need before you start a job.

3. **Diagnose the Problem Accurately:** Before attempting any repair, make sure you accurately diagnose the issue. This could involve checking error codes with an OBD-II scanner, listening for unusual sounds, or visually inspecting components for damage or wear.

4. **Use Online Resources and Tutorials:** There are numerous online resources available, including forums, YouTube videos, and repair guide websites 

You can start the visualisation front end by entering the following command in the terminal, then open the link shown in the output in your browser.

> ell-studio --storage ./logdir

The following example uses Pydantic Python which allows us to specify the structure/schema of content.  Importantly LLMs can understand these structures and it becomes a very useful way for us to specify the format it should return the response in.

The file movies.py has the layout of the response we want the LLM to return.

https://docs.pydantic.dev/latest/

In [19]:
# Redoing the previous movies request with Pydantic python defining the output structure.

import importlib
import movies
importlib.reload(movies)

# Generate a movie review
message = movies.generate_movie_review("Dirty Dozen")
review = message.parsed

# Debug print
#movies.print_review_debug(review)

# Access individual fields
print(f"Movie Title: {review.title}")
print(f"Rating: {review.rating}/10")
print(f"Summary: {review.summary}")
print(f"Cast:")
for actor in review.cast:
    print(f"\t- {actor.name}")

Movie Title: The Dirty Dozen
Rating: 9/10
Summary: The Dirty Dozen is a gripping World War II action film that follows a group of 12 convicted soldiers given a chance at redemption. Assembled for a suicide mission behind enemy lines, they must infiltrate a Nazi stronghold to eliminate high-rank German officers. With intense action sequences and a touch of dark humor, this classic delivers thrilling entertainment.
Cast:
	- Lee Marvin
	- Ernest Borgnine
	- Charles Bronson
	- Jim Brown
	- John Cassavetes
